In [30]:
import logging
import sys
from pathlib import Path
import itertools

import click
import numpy as np
import pandas as pd
import rasterra as rt
from rasterio.features import rasterize
from rra_tools import jobmon
import xarray as xr
import tqdm
import geopandas as gpd

from spatial_temp_cgf import paths, utils
from spatial_temp_cgf.data_prep.location_mapping import FHS_SHAPE_PATH
from spatial_temp_cgf import cli_options as clio
from spatial_temp_cgf.data import DEFAULT_ROOT, ClimateMalnutritionData

In [2]:
output_dir = Path(DEFAULT_ROOT)
measure = 'stunting'
model_version = "2024_07_01.13"
cmip6_scenario = 'ssp126'
year = 2025

In [3]:
cm_data = ClimateMalnutritionData(output_dir / measure)
spec = cm_data.load_model_specification(model_version)
print('loading raster template and shapes')
raster_template = cm_data.load_raster_template()
fhs_shapes = cm_data.load_fhs_shapes()
print('loading models')
models = cm_data.load_model_family(model_version)

loading raster template
loading models


In [19]:
m = models[0]

In [13]:
raster_template

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [31]:
coefs = m['model'].coefs['Estimate']
refs = m['model'].ranef

p = spec.predictors[0]

def get_intercept_raster(model, param_spec):
    coefs = model['model'].coefs['Estimate']
    refs = model['model'].ranef
    icept = coefs.loc['(Intercept)']
    if p.random_effect == 'ihme_loc_id':
        r_icept = refs['X.Intercept.'].reset_index()
        r_icept = r_icept.merge(fhs_shapes, left_on='index', right_on='ihme_lc_id', how='left')
        shapes = list(r_icept[['geometry', 'X.Intercept.']].itertuples(index=False, name=None))
        icept_arr = rasterize(
            shapes, out=icept*np.ones_like(raster_template), transform=raster_template.transform, 
        )
        icept_raster = rt.RasterArray(
            icept_arr,
            transform=raster_template.transform,
            crs=raster_template.crs,
            no_data_value=np.nan
        )
    elif not p.random_effect:
        icept_raster = raster_template + icept
    else:
        msg = 'Only location random intercepts are supported'
        raise NotImplementedError(msg)
    
    
    
    

In [33]:
icept_raster

RasterArray
dimensions    : 36000, 18000 (x, y)
resolution    : 0.01, -0.01 (x, y)
extent        : -180.0, 180.0, -90.0, 90.0 (xmin, xmax, ymin, ymax)
crs           : EPSG:4326
no_data_value : nan
size          : 2471.92 MB
dtype         : float32

In [20]:
refs

,X.Intercept.
AGO,0.343156
ALB,-0.308619
ARM,-0.059929
BDI,0.746451
BEN,0.153004
BFA,-0.262447
BOL,-0.539367
CAF,-0.048215
CIV,0.058785
CMR,-0.114749
